# Création automatique des tables de dimensions

In [3]:
from scripts.config import *
import glob
import pandas as pd
from tqdm import tqdm

## Fixing root path issues in `config.py`

In [4]:
import os
import sys

os.path.dirname(os.getcwd())
os.PathLike()

AttributeError: module 'os' has no attribute 'PathLike'

## Resolve files name pattern from upload

La solution la plus rapide est d'utiliser une regex et la librairie glob.

- `Bloc*.xls*` pour les données des machines ex: __Bloc1D4G.xls__
- `base LgTx *.xls*` pour les données des patients et donneur ex: __base LgTx IA 24oct2018__

In [6]:
path_to_data = DATA_ORIGINAL_MACHINES

files = glob.glob('{}{}'.format(path_to_data,FILE_NAME_PATTERN['machines']))

print("Found {} files".format(len(files)))
files

Found 8 files


['/Users/yanni-benoit-iyeze/Documents/Data For Good/batch_5_transplant/data/original_data/machines/Bloc3D4G.xls',
 '/Users/yanni-benoit-iyeze/Documents/Data For Good/batch_5_transplant/data/original_data/machines/Bloc1D4G.xls',
 '/Users/yanni-benoit-iyeze/Documents/Data For Good/batch_5_transplant/data/original_data/machines/Bloc9D4G.xls',
 '/Users/yanni-benoit-iyeze/Documents/Data For Good/batch_5_transplant/data/original_data/machines/Bloc7D4G.xls',
 '/Users/yanni-benoit-iyeze/Documents/Data For Good/batch_5_transplant/data/original_data/machines/Bloc2D4G.xls',
 '/Users/yanni-benoit-iyeze/Documents/Data For Good/batch_5_transplant/data/original_data/machines/Bloc4D4G.xls',
 '/Users/yanni-benoit-iyeze/Documents/Data For Good/batch_5_transplant/data/original_data/machines/Bloc8D4G.xls',
 '/Users/yanni-benoit-iyeze/Documents/Data For Good/batch_5_transplant/data/original_data/machines/Bloc6D4G.xls']

In [9]:
path_to_data = DATA_ORIGINAL_PATIENT

print(FILE_NAME_PATTERN['patient-donor'])
files = glob.glob('{}{}'.format(path_to_data,FILE_NAME_PATTERN['patient-donor']))

print("Found {} files".format(len(files)))
files


base*.xlsx
Found 1 files


['/Users/yanni-benoit-iyeze/Documents/Data For Good/batch_5_transplant/data/original_data/patient-donor/base LgTx IA 24oct2018.xlsx']

In [16]:
file = pd.ExcelFile(files[0])
output = {}
for sheet in tqdm(file.sheet_names):
    if sheet not in ['ensemble', 'glossaire']:
        df = pd.read_excel(files[0], sheet, index_col=None)
        if sheet != 'donneur':
            sheet = '{}_{}'.format('patient', sheet)
        
        filename = os.path.join(DATA_MODEL, 'dim_{}.csv'.format(sheet))
        df.to_csv(filename)
        info = {
        sheet : list(df.columns.values)
        }
        output.update(info)
DIM_MAP_FIELDS = output
output

100%|██████████| 6/6 [00:12<00:00,  2.06s/it]


{'donneur': ['numero',
  'CMV_donneur',
  'EBV_donneur',
  'Age_donor',
  'Sex_donor',
  'BMI_donor',
  'Poids_donor',
  'Taille_donor',
  'Donneur_CPT',
  'Tabagisme_donor',
  'Aspirations_donor',
  'RX_donor',
  'PF_donor',
  'oto_score'],
 'patient_intraoperatoire': ['numero',
  'exvivo',
  'Immunosuppresseurs',
  'heure_transfert_rea',
  'date_sortie_bloc',
  'duree_sejour_bloc',
  'Antibioprophylaxie',
  'PB_induction',
  'Pb_induction_detail',
  'FIO2_initiale',
  'pH_initial',
  'PAPS_initiale',
  'PA_initiale',
  'VT_initial',
  'PaCO2_initial',
  'PAPM_initiale',
  'Fc_initiale',
  'FR_initial',
  'PaO2_initial',
  'PAPD_initiale',
  'NORAD_initiale',
  'PEEP_initial',
  'Bicarbonates_initial',
  'NO_initiale',
  'Lactate_initial',
  'SvO2_initiale',
  'Ic_initial',
  'Hb_initial',
  'Qc_initiale',
  'Examen_Echographique_initial',
  'Problemes_VBP',
  'Problemes_VBP_commentaires',
  'premiere_transplantation_cote',
  'Ventilation_Unipulmonaire_Pb',
  'FiO2_clampage_cote_1',
 

In [13]:
for name, array in DIM_MAP_FIELDS.items():
    df_ = df[df.columns.intersection(array)]
    filename = os.path.join('', 'dim_{}.csv'.format(name))
    #df.to_csv(filename)

donneur ['numero', 'CMV_donneur', 'EBV_donneur', 'Age_donor', 'Sex_donor', 'BMI_donor', 'Poids_donor', 'Taille_donor', 'Donneur_CPT', 'Tabagisme_donor', 'Aspirations_donor', 'RX_donor', 'PF_donor', 'oto_score']
patient_postoperatoire ['numero', 'LOS_first_ventilation', 'LOS_total_ventilation', ' immediate_extubation', 'secondary_intubation', 'time_to_secondary_intubation', 'secondary_ECMO', 'delai_recours_ECMO', 'Cause_ECMO_secondaire', 'postoperative_vasopressive_support', 'ACFA', 'PRBC_postop', 'FFP_postop', 'Platelets_postop', 'CVA', 'hemodyalisis', 'tracheostomy', 'reoperation_for_bleeding', 'bleeding', 'lower_limb_complication', 'lower_limb_ischemia', 'scarpa_complication', 'vascular_complications', 'thromboembolic_complication', 'choc_septique', 'cardiac_arrest_during_surgery', 'LOS_ICU', 'LOS_hosp', 'in_hospital_mortality', '30_d_survival', 'P_F_H0', 'PGD_H0', 'P_F_end_surgery', 'PGD_end_surgery', 'time_last_PF', 'PDG_h24', 'PGD_h48', 'PGD3', 'date_de_deces', 'Survival_days_27_1

Déterminer les colonnes partagées par les différentes tables de dimensions.

In [15]:
df_intra = pd.read_csv('dim_patient_intraoperatoire.csv')
df_post = pd.read_csv('dim_patient_postoperatoire.csv')
df_pre = pd.read_csv('dim_patient_preoperatoire.csv')

In [14]:
common_columns = df_intra.columns & df_pre.columns & df_post.columns
print(common_columns)
print("Nb of common columns: {}".format(common_columns.size))

Index(['Unnamed: 0', 'numero', 'LOS_first_ventilation',
       'LOS_total_ventilation', ' immediate_extubation',
       'secondary_intubation', 'time_to_secondary_intubation',
       'secondary_ECMO', 'delai_recours_ECMO', 'Cause_ECMO_secondaire',
       'postoperative_vasopressive_support', 'ACFA', 'PRBC_postop',
       'FFP_postop', 'Platelets_postop', 'CVA', 'hemodyalisis', 'tracheostomy',
       'reoperation_for_bleeding', 'bleeding', 'lower_limb_complication',
       'lower_limb_ischemia', 'scarpa_complication', 'vascular_complications',
       'thromboembolic_complication', 'choc_septique',
       'cardiac_arrest_during_surgery', 'LOS_ICU', 'LOS_hosp',
       'in_hospital_mortality', '30_d_survival', 'P_F_H0', 'PGD_H0',
       'P_F_end_surgery', 'PGD_end_surgery', 'time_last_PF', 'PDG_h24',
       'PGD_h48', 'PGD3', 'date_de_deces', 'Survival_days_27_10_2018'],
      dtype='object')
Nb of common columns: 41


In [1]:
from scripts.parser import Parser
parser = Parser()

parser.update_tables()

Found 1 files ... 
Importing and cleaning Excel files ...
Uploading sheets of the Excel file ...


100%|██████████| 6/6 [00:12<00:00,  2.00s/it]


Found 13 files ... 
Importing and cleaning Excel files ...


100%|██████████| 13/13 [01:06<00:00,  5.46s/it]


Exporting clean file in  /Users/yanni-benoit-iyeze/Documents/Data For Good/batch_5_transplant/data/clean_data/...
Creating the data model in /Users/yanni-benoit-iyeze/Documents/Data For Good/batch_5_transplant/data/clean_data/model/...
